This notebook aims to contain all functions for indicators.

In [ ]:
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import gumbel_r
from scipy.stats import gumbel_l
import os
import os.path
import math

# Precipitation

In [2]:
# import data

In [3]:
out_path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6'
csv_path = os.path.join(out_path,'csv_file','pr_day_2021-2060','All_projects_moz_precipitation_2021-2060_without_month_year.csv')
precipitation_2021_2060 = pd.read_csv(csv_path,header=[0,1],index_col=[0,1,2,3,4])*86400 # units kg.m^(-2).s^(-1)
# convert precipitation data from kg.m^(-2).s^(-1) to mm/day :  1 kg/m2/s = 86400 mm/day
# source: https://www.researchgate.net/post/How-do-I-convert-ERA-Interim-precipitation-estimates-from-kg-m2-s-to-mm-day
list_time_2021_2060=pd.date_range('01-01-2021','31-12-2060', freq='D').strftime('%d-%m-%Y').values.tolist()

### Return period

In [28]:
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True)
precipitation_2021_2060_copy.drop('NESM3', level=2, axis=0, inplace=True)
precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]

C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_172424\2322686404.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],'NESM3')]


KeyError: ('Chimoio_2ML_Reservoir', 'ssp126', 'NESM3')

In [29]:
precipitation_2021_2060_copy

Longitude  \
                                                                    33.625   
Name project          Experiment Model      Date       Latitude              
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125    5.057179   
                                            02-01-2021 -16.125   17.624178   
                                            03-01-2021 -16.125    7.632648   
                                            04-01-2021 -16.125    4.736492   
                                            05-01-2021 -16.125    4.981353   
...                                                                    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125         NaN   
                                            28-12-2060 -13.125         NaN   
                                            29-12-2060 -13.125         NaN   
                                            30-12-2060 -13.125         NaN   
                                            31-12-2060 -13.125         NaN   

                                                                               \
                                                                39.875 36.875   
Name project          Experiment Model      Date       Latitude                 
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125     NaN    NaN   
                                            02-01-2021 -16.125     NaN    NaN   
                                            03-01-2021 -16.125     NaN    NaN   
                                            04-01-2021 -16.125     NaN    NaN   
                                            05-01-2021 -16.125     NaN    NaN   
...                                                                ...    ...   
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125     NaN    NaN   
                                            28-12-2060 -13.125     NaN    NaN   
                                            29-12-2060 -13.125     NaN    NaN   
                                            30-12-2060 -13.125     NaN    NaN   
                                            31-12-2060 -13.125     NaN    NaN   

                                                                           
                                                                   40.375  
Name project          Experiment Model      Date       Latitude            
PT_Revubue_2_Rev_2_01 ssp245     ACCESS-CM2 01-01-2021 -16.125        NaN  
                                            02-01-2021 -16.125        NaN  
                                            03-01-2021 -16.125        NaN  
                                            04-01-2021 -16.125        NaN  
                                            05-01-2021 -16.125        NaN  
...                                                                   ...  
PT__Dumping_Site      ssp370     TaiESM1    27-12-2060 -13.125   0.000000  
                                            28-12-2060 -13.125   0.435942  
                                            29-12-2060 -13.125   6.300328  
                                            30-12-2060 -13.125   8.168522  
                                            31-12-2060 -13.125   0.737792  

[5727120 rows x 4 columns]

In [30]:
# treatment of dataframe to make useable
df_years=precipitation_2021_2060_copy.copy(deep=True)
#df_year = df_year1.loc[('PT_Revubue_2_Rev_2_01')]
df_years.columns = df_years.columns.droplevel(0) # drop first level of column name
df_years['33.625'] = df_years['33.625'].fillna(0)
df_years['39.875'] = df_years['39.875'].fillna(0)
df_years['36.875'] = df_years['36.875'].fillna(0)
df_years['40.375'] = df_years['40.375'].fillna(0)
df_years['33.625'] = df_years['33.625'] + df_years['39.875'] + df_years['36.875'] + df_years['40.375']
df_years = df_years[['33.625']]
df_years=df_years.rename(columns={df_years.columns[0]:'Precipitation mm/day'})
df_years=df_years.droplevel(level=4) # drop latitude index
df_years=df_years.reset_index()

Year = df_years[['Date']].values.reshape(len(df_years[['Date']].values),)
for i in np.arange(0,len(df_years[['Date']].values)):
    Year[i]=str(Year[i][6:10])

df_years['Year'] = Year
df_years=df_years.drop(labels='Date',axis=1)

df_years_max = df_years.groupby(['Name project','Experiment','Model','Year']).max() # maximum

In [31]:
df_years_max

Precipitation mm/day
Name project          Experiment Model      Year                      
Chimoio_2ML_Reservoir ssp126     ACCESS-CM2 2021             22.153287
                                            2022             45.533448
                                            2023             23.700950
                                            2024             32.253806
                                            2025             21.849809
...                                                                ...
WTP_gate_cnr1         ssp585     TaiESM1    2056             50.823568
                                            2057             33.281169
                                            2058             18.805485
                                            2059             19.757411
                                            2060             20.890592

[15680 rows x 1 columns]

In [32]:
# return period for each project, model, scenario

In [33]:
# function value for return period

In [34]:
def value_return_period(Z,T):
    (loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
    # gumbel_r is chosen because
    p_non_exceedance = 1 - (1/T)
    try:
        threshold_coresponding = round(gumbel_r.ppf(p_non_exceedance,loc,scale))
    except OverflowError: # the result is not finite
        if math.isinf(gumbel_r.ppf(p_non_exceedance,loc,scale)) and gumbel_r.ppf(p_non_exceedance,loc,scale)<0:
            # the result is -inf
            threshold_coresponding = 0 # the value of wero is imposed
    return threshold_coresponding
    # ppf: Percent point function
    #print('Threshold '+str(threshold_coresponding)+' mm/day will be exceeded at least once in '+str(n)+' year, with a probability of '+str(round(p_exceedance*100))+ ' %')
    #print('This threshold corresponds to a return period of '+str(round(return_period))+ ' year event over a '+str(n)+' year period')

In [40]:
models=precipitation_2021_2060_copy.index.levels[2].tolist()

In [41]:
models.remove('NESM3')

In [42]:
# create empty dataframe to register return value period
midx = pd.MultiIndex.from_product([precipitation_2021_2060_copy.index.levels[0].tolist(),precipitation_2021_2060_copy.index.levels[1].tolist(),models],names=['Name project','Experiment', 'Model'])
cols = ['Return period 50 years','Return period 100 years']
return_period = pd.DataFrame(data = [], 
                            index = midx,
                            columns = cols)
    #return_period = pd.concat([return_period,Variable_dataframe])

In [43]:
for name_p in return_period.index.levels[0].tolist():
    for ssp in return_period.index.levels[1].tolist():
        for model in return_period.index.levels[2].tolist():
            print('Name project '+name_p+ ' ssp '+ssp+ ' model '+model)
            Z=df_years_max.loc[(name_p,ssp,model)].values.reshape(40,)
            return_period.loc[(name_p,ssp,model),('Return period 50 years')] = value_return_period(Z,50)
            return_period.loc[(name_p,ssp,model),('Return period 100 years')] = value_return_period(Z,100)

Name project Chimoio_2ML_Reservoir ssp ssp126 model ACCESS-CM2
Name project Chimoio_2ML_Reservoir ssp ssp126 model ACCESS-ESM1-5
Name project Chimoio_2ML_Reservoir ssp ssp126 model BCC-CSM2-MR
Name project Chimoio_2ML_Reservoir ssp ssp126 model CMCC-CM2-SR5
Name project Chimoio_2ML_Reservoir ssp ssp126 model CMCC-ESM2
Name project Chimoio_2ML_Reservoir ssp ssp126 model CanESM5
Name project Chimoio_2ML_Reservoir ssp ssp126 model IITM-ESM
Name project Chimoio_2ML_Reservoir ssp ssp126 model MIROC6
Name project Chimoio_2ML_Reservoir ssp ssp126 model MPI-ESM1-2-HR
Name project Chimoio_2ML_Reservoir ssp ssp126 model MPI-ESM1-2-LR
Name project Chimoio_2ML_Reservoir ssp ssp126 model MRI-ESM2-0
Name project Chimoio_2ML_Reservoir ssp ssp126 model NorESM2-LM
Name project Chimoio_2ML_Reservoir ssp ssp126 model NorESM2-MM
Name project Chimoio_2ML_Reservoir ssp ssp126 model TaiESM1
Name project Chimoio_2ML_Reservoir ssp ssp245 model ACCESS-CM2
Name project Chimoio_2ML_Reservoir ssp ssp245 model ACCE

Name project PT_Revubue_2_Rev_2_01 ssp ssp370 model MRI-ESM2-0
Name project PT_Revubue_2_Rev_2_01 ssp ssp370 model NorESM2-LM
Name project PT_Revubue_2_Rev_2_01 ssp ssp370 model NorESM2-MM
Name project PT_Revubue_2_Rev_2_01 ssp ssp370 model TaiESM1
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model ACCESS-CM2
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model ACCESS-ESM1-5
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model BCC-CSM2-MR
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model CMCC-CM2-SR5
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model CMCC-ESM2
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model CanESM5
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model IITM-ESM
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model MIROC6
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model MPI-ESM1-2-HR
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model MPI-ESM1-2-LR
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model MRI-ESM2-0
Name project PT_Revubue_2_Rev_2_01 ssp ssp585 model NorE

In [44]:
return_period

Return period 50 years  \
Name project          Experiment Model                                  
Chimoio_2ML_Reservoir ssp126     ACCESS-CM2                        47   
                                 ACCESS-ESM1-5                     48   
                                 BCC-CSM2-MR                       62   
                                 CMCC-CM2-SR5                      48   
                                 CMCC-ESM2                         62   
...                                                               ...   
WTP_gate_cnr1         ssp585     MPI-ESM1-2-LR                     51   
                                 MRI-ESM2-0                        56   
                                 NorESM2-LM                        44   
                                 NorESM2-MM                        54   
                                 TaiESM1                           70   

                                               Return period 100 years  
Name project          Experiment Model                                  
Chimoio_2ML_Reservoir ssp126     ACCESS-CM2                         51  
                                 ACCESS-ESM1-5                      51  
                                 BCC-CSM2-MR                        67  
                                 CMCC-CM2-SR5                       51  
                                 CMCC-ESM2                          68  
...                                                                ...  
WTP_gate_cnr1         ssp585     MPI-ESM1-2-LR                      56  
                                 MRI-ESM2-0                         61  
                                 NorESM2-LM                         47  
                                 NorESM2-MM                         58  
                                 TaiESM1                            77  

[392 rows x 2 columns]

In [59]:
# dataframe contains statistics about futur value for return period
return_period.astype(int).groupby(['Name project']).describe(percentiles=[.1, .5, .9]) # astype(int) necessary to have stats min, max and percentiles

Return period 50 years                             \
                                            count       mean       std   min   
Name project                                                                   
Chimoio_2ML_Reservoir                        56.0  54.035714  5.685594  45.0   
Mugica_dam_road                              56.0  59.392857  6.409307  47.0   
PT1_Pemba__Breidge_Paquite                   56.0  53.857143  6.947671  38.0   
PT_Nicoadala_F5                              56.0  72.642857  9.387474  54.0   
PT_Revubue_2_Rev_2_01                        56.0  52.428571  7.743284  40.0   
PT__Dumping_Site                             56.0  53.232143  7.165552  39.0   
WTP_gate_cnr1                                56.0  52.428571  7.743284  40.0   

                                                    Return period 100 years  \
                             10%   50%   90%    max                   count   
Name project                                                                  
Chimoio_2ML_Reservoir       47.0  54.0  61.5   66.0                    56.0   
Mugica_dam_road             51.5  60.0  67.0   79.0                    56.0   
PT1_Pemba__Breidge_Paquite  47.0  55.0  62.0   68.0                    56.0   
PT_Nicoadala_F5             63.0  71.0  86.5  101.0                    56.0   
PT_Revubue_2_Rev_2_01       43.0  53.0  61.5   78.0                    56.0   
PT__Dumping_Site            45.0  53.0  63.0   71.0                    56.0   
WTP_gate_cnr1               43.0  53.0  61.5   78.0                    56.0   

                                                                          \
                                 mean        std   min   10%   50%   90%   
Name project                                                               
Chimoio_2ML_Reservoir       58.464286   6.390232  49.0  51.0  58.5  67.0   
Mugica_dam_road             64.517857   7.331107  51.0  55.5  65.0  73.0   
PT1_Pemba__Breidge_Paquite  58.642857   7.824039  41.0  51.0  60.0  68.0   
PT_Nicoadala_F5             78.714286  10.618533  58.0  67.5  76.5  94.5   
PT_Revubue_2_Rev_2_01       56.982143   8.859491  43.0  46.0  57.0  68.0   
PT__Dumping_Site            57.946429   8.064331  42.0  48.5  57.0  69.0   
WTP_gate_cnr1               56.982143   8.859491  43.0  46.0  57.0  68.0   

                                   
                              max  
Name project                       
Chimoio_2ML_Reservoir        73.0  
Mugica_dam_road              88.0  
PT1_Pemba__Breidge_Paquite   75.0  
PT_Nicoadala_F5             110.0  
PT_Revubue_2_Rev_2_01        86.0  
PT__Dumping_Site             77.0  
WTP_gate_cnr1                86.0

In [ ]:
#Z = test['Precipitation mm/day'].values
Z.sort()
(loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
# xaxis is precipitation and yaxis is densiy of probability
myHist = plt.hist(Z,density=True) # If ``True``, draw and return a probability density: each bin 
# will display the bin's raw count divided by the total number of counts *and the bin width*
h = plt.plot(Z,gumbel_r.pdf(Z,loc,scale))
plt.xlabel('Precipitation value mm/day')
plt.ylabel('Density of probability' )
plt.title('Histogram and probability density function of precipitation values\nfor year 2021 for one project ,\n one scenario and one model',fontdict={'fontsize': 10})
plt.legend(['Probability density function','Histogramm'])
title_png = 'test_density.png'
path_figure = os.path.join(out_path,'figures')
if not os.path.isdir(path_figure):
    os.makedirs(path_figure)
#plt.savefig(os.path.join(path_figure,title_png),format ='png')
plt.show()

In [ ]:
# accross models and scenarios

In [ ]:
test=precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0])]
test=test[[('Longitude','36.875')]]
test=test.droplevel(level=3) # drop latitude index
test.columns = test.columns.droplevel(0) # drop first level of column name
test=test.rename(columns={test.columns[0]:'Precipitation mm/day'})
test=test.swaplevel(0,2,axis=0)
test = test.filter(like = str(2021), axis=0) # select only data for one year
#test#['Precipitation mm'].values
test.drop(labels='NESM3',level=1,inplace=True)

In [ ]:
Z = test['Precipitation mm/day'].values
Z.sort()
(loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
# xaxis is precipitation and yaxis is densiy of probability
myHist = plt.hist(Z,density=True) # If ``True``, draw and return a probability density: each bin 
# will display the bin's raw count divided by the total number of counts *and the bin width*
h = plt.plot(Z,gumbel_r.pdf(Z,loc,scale))
plt.xlabel('Precipitation value mm/day')
plt.ylabel('Density of probability' )
plt.title('Histogram and probability density function of precipitation values\nfor year 2021 for one project ,\n with all scenarios and models',fontdict={'fontsize': 10})
plt.legend(['Probability density function','Histogramm'])
title_png = 'test_density.png'
path_figure = os.path.join(out_path,'figures')
if not os.path.isdir(path_figure):
    os.makedirs(path_figure)
#plt.savefig(os.path.join(path_figure,title_png),format ='png')
plt.show()

In [ ]:
precipitation_2021_2060_copy.index.levels[0]

In [ ]:
# questions Temps retour :
#      tjs avec maximum ? oui, proba sur un an
#      besoin de caler mieux distribution ? package pour le faire automatiquement ? si on fiat pas avec maxima, mais on va faire que avec maxima pour le moment

### N-day event 

In [64]:
# some models to not have any values for some scenarios
# need to delete them from the global dataset

# PROBLEM AVEC CETTE FUNCTION

def delete_NaN_model(df):
    df_copy=df.copy(deep=True) # copy the original dataframe, not to modify the original one    
    model_to_delete =[]
    longitude=[]
    for project in df_copy.index.levels[0].tolist(): # projects
        # look value of longitude for each project
        for j in np.arange(0,len(df_copy.loc[[project]].columns)):
            if ~df_copy[[df_copy.columns[j]]].isnull().values.all():
                longitude.append(df_copy.columns[j])
                continue
        
        for scenario in df_copy.index.levels[1].tolist(): # scenarios
            for model in df_copy.index.levels[2].tolist(): # models
                if df_copy.loc[(project,scenario, model)].isnull().values.all():
                    print('No data for Project '+ project+', scenario '+scenario+', model '+model)
                    # all the values for the given project, scenario and model are NaN
                    if model not in model_to_delete:
                        model_to_delete.append(model)# keep unique values
    
    if model_to_delete!=[]:
        # for some given project, scenario and model, there is no values
        for model in model_to_delete:
            models_index = df_copy.index.levels[2].tolist()
            models_index.remove(model)
            df_copy.drop(labels=model,level=2,inplace=True)
        
        return models_index
        # create new dataframe with correct index
    return []

In [60]:
# this functions aims to calculate the n_day_event
def n_day_maximum_rainfall(number_day,df):
    df1=df.copy(deep=True)
    # df.use function rolling(n).sum() to calculate cumulative precipitation over n days
    df1[['Precipitation mm']]=df1[['Precipitation mm']].rolling(number_day).sum()
    time=df1.index.tolist()
    for k in np.arange(len(time)-number_day,-1,-1):
        time[number_day-1+k] = time[k] + ' to '+time[number_day-1+k]
    df1.drop(df1.index[np.arange(0,number_day-1)], inplace=True) # delete first elements which are NaNs
    del time[0:number_day-1] # delete firsts elements, which have no value associated with
    #midx = pd.MultiIndex.from_product([ time],names=['Date'])
    name_col = ['Precipitation mm']
    Dataframe_n_day_event = pd.DataFrame(data = df1.values, 
                                index = [time],
                                columns = name_col)
    return Dataframe_n_day_event

In [61]:
# this function aims to create the empty dataframe that will be filled

def fill_dataframe(name_project,scenario,model,time,data_df):
    #df = pd.DataFrame()
    for i in np.arange(0,len(name_project)):
        midx = pd.MultiIndex.from_product([(name_project,),(scenario,),(model,) , time],names=['Name project','Experiment', 'Model', 'Date'])
        name_col = ['Precipitation mm']#['Precipitation '+str(number_day)+' day event mm']
        Variable_dataframe = pd.DataFrame(data = data_df, 
                                    index = midx,
                                    columns = name_col)
        #df = pd.concat([df,Variable_dataframe])
    return Variable_dataframe

In [62]:
# function dataframe_n_day_event produce a dataframe, with the n_day event precipitation for the period, models and scenarios asked
# this function use the function : 'delete_NaN_model', 'n_day_maximum_rainfall' and 'fill_dataframe'


def dataframe_n_day_event(df,number_day):
    df_copy=df.copy(deep=True) # copy the original dataframe, not to modify the original one    
    df_n_day_event = pd.DataFrame() # create empty dataframe, that will be filled later
    # extract years of the period of interest, make a vector containing all the years of interest
    years = np.arange(int(df.index.levels[3].tolist()[0][6:10]),int(df.index.levels[3].tolist()[len(df.index.levels[3].tolist())-1][6:10])+1)
    models_index=delete_NaN_model(precipitation_2021_2060) # use function 'delete_NaN_model' to know which models have no Nan values
    df_copy=df_copy.droplevel(level=4) # drop latitude index
    df_copy.columns = df_copy.columns.droplevel(0) # drop first level of column name
    for project in df_copy.index.levels[0].tolist(): # projects
        for scenario in df_copy.index.levels[1].tolist(): # scenarios
            for model in models_index: # models
                print('Project '+ project+', scenario '+scenario+', model '+model)
                # select on project, one scenario, one model and drop Latitude index
                df_temp_all_years = df_copy.loc[(project,scenario,model)]
                # find which columns does not only have NaN
                for j in np.arange(0,len(df_temp_all_years.columns)): # for loop to have number of the column
                    if ~df_temp_all_years[[df_temp_all_years.columns[j]]].isnull().values.all():
                        # the column does not only have Nan values
                        df_temp_all_years=df_temp_all_years[[df_temp_all_years.columns[j]]] # register only column with values, and not the NaN values
                        df_temp_all_years=df_temp_all_years.rename(columns={df_temp_all_years.columns[0]:'Precipitation mm'})
                        # rename the column
                        break # stop the for loop with the number of columns, because values were found
                        # go to line if df_temp_all_years.columns.nlevels!=1:
                if df_temp_all_years.columns.nlevels!=1:
                    # the dataframe still has two levels of columns, so the precedent if condition was never fullfilled
                    print('The model '+model+' has no data')
                    continue # try with the next model
                else:
                    # the dataframe still has one level of columns, there was one column not containing only NaN values
                    for year in years:
                        print(year)
                        df_temp_one_year = df_temp_all_years.filter(like = str(year), axis=0) # select only data for one year
                        #return df_temp_one_year
                        df_temp_one_year_n_event=n_day_maximum_rainfall(number_day,df_temp_one_year) # use function to calculate cumulative precipitation
                        #return df_temp_one_year_n_event
                        # format time vector differently
                        time = [df_temp_one_year_n_event.index.tolist()[i][0] for i in np.arange(0,len(df_temp_one_year_n_event.index.tolist()))]
                        # fill dataframe
                        df_temp_one_year_n_event = fill_dataframe(project,scenario,model,time,df_temp_one_year_n_event.values)
                        df_n_day_event = pd.concat([df_n_day_event,df_temp_one_year_n_event])
    return df_n_day_event # return a dataframe, with all the projects, scenarios, models and period of n day

In [ ]:
df_5_day_event=dataframe_n_day_event(precipitation_2021_2060_copy,5) # checker si ca joue. a commencer a 10h19

C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_172424\2866413072.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  if df_copy.loc[(project,scenario, model)].isnull().values.all():


No data for Project Chimoio_2ML_Reservoir, scenario ssp370, model NESM3


C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_172424\2866413072.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  if df_copy.loc[(project,scenario, model)].isnull().values.all():


No data for Project Mugica_dam_road, scenario ssp370, model NESM3


C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_172424\2866413072.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  if df_copy.loc[(project,scenario, model)].isnull().values.all():


...

2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
Project PT_Nicoadala_F5, scenario ssp245, model ACCESS-ESM1-5
2021


C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_172424\1793876292.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_temp_all_years = df_copy.loc[(project,scenario,model)]


2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
Project PT_Nicoadala_F5, scenario ssp245, model BCC-CSM2-MR
2021


C:\Users\CLMRX\AppData\Local\Temp\1\ipykernel_172424\1793876292.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_temp_all_years = df_copy.loc[(project,scenario,model)]


2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033


In [ ]:
# find max per year

### Yearly average precipitation

### Seasonal average precipitation